In [1]:
%%writefile app.py
import os
import pandas as pd
import numpy as np
from flask import Flask, flash, request, redirect, url_for, send_from_directory, render_template
from werkzeug.utils import secure_filename
import sqlite3
import query_alerts
from sqlalchemy import create_engine, text
from sqlalchemy.sql import select

app = Flask(__name__)

# Write a function to view lightcurves (click and can go back to list, or go "next"), with an option to sort.
# Write a function to query and sort. 
# Option to save querys? 
# How to do version control? (As alert values update.) Have multiple databases? 
# Save in the database name the day it was created? 
# Have some function that knows the current year. If it was an old year, this is unnecessary.

engine = create_engine('sqlite:///foo.db')
conn = engine.connect()

@app.route('/', methods=['GET', 'POST'])
def start_page():
    """
    Homepage. Depending on whether there is a database or not,
    will either prompt you to upload, or will give you the 
    option to upload or query.
    """
    if engine.table_names() == []:
        return render_template('start_empty.html', web_download_to_db=url_for('web_download_to_db'))

    else:
        return render_template('start_filled.html', 
                               web_download_to_db=url_for('web_download_to_db'), 
                               query_db=url_for('query_db'),
                               dbs=engine.table_names())

@app.route('/update', methods=['GET', 'POST'])
def web_download_to_db():
    """
    FIXME: add an option to choose whether to keep or delete database.
    Actually, have radio buttons for this: Choose whether want lightcurves, data, system, and years.

    """
    
    # FIXME: Check if database exists, choose to overwrite or not (based on current year)
    # FIX: Not sure if get_kmtnet_lightcurves works right.
    
    # Check if database exists.
    return render_template('download_data.html', start_page=url_for('start_page'))
    # return render_template('example.html')
    
    # FINISH THIS
    # if key in engine.table_names()

@app.route('/handle_data', methods=['GET', 'POST'])
def handle_data():
    """
    FIXME: add an option to choose whether to keep or delete database.
    Actually, have radio buttons for this: Choose whether want lightcurves, data, system, and years.

    """
    keys = list(request.form.to_dict().keys())
    for ii, key in enumerate(keys):
        system, data, year = key.split('_')
        
        if system == 'KMTNet':
            if data == 'alerts':
                query_alerts.get_kmtnet_alerts(year)
            else: # lightcurves
                query_alerts.get_kmtnet_lightcurves(year)
        elif system == 'OGLE':
            if data == 'alerts':
                query_alerts.get_ogle_alerts(year)
            else: # lightcurves
                query_alerts.get_ogle_lightcurves(year)
        else: # MOA
            if data == 'alerts':
                query_alerts.get_moa_alerts(year)
            else: # lightcurves
                query_alerts.get_moa_lightcurves(year)
    
    # How to show processing?
    return render_template('start_filled.html', 
                            web_download_to_db=url_for('web_download_to_db'), 
                            query_db=url_for('query_db'))

@app.route('/query', methods=['GET', 'POST'])
def query_db():
    """
    Provide the interface to query the database.
    """
    if request.method == 'POST':
        db_info = engine.execute(request.form['query']).fetchall()
        
        return render_template('display.html', len = len(db_info), 
                               db_info = db_info, 
                               start_page=url_for('start_page'))

    return render_template('query.html', start_page=url_for('start_page'), 
                            dbs=engine.table_names())

if __name__ == '__main__':
    app.run(port=8000, debug = True)

Overwriting app.py


In [ ]:
!python app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 145-043-719
/u/casey/scratch/code/python-ay250-homeworks/final_project/app.py:102: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  dbs=engine.table_names())
127.0.0.1 - - [03/May/2022 15:58:13] "GET /query HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2022 15:59:21] "POST /query HTTP/1.1" 200 -
/u/casey/scratch/code/python-ay250-homeworks/final_project/app.py:30: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  if engine.table_names() == []:
/u/casey/scratch/code/python-ay250-homeworks/